In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from collections import Counter
from sklearn.metrics import log_loss
import xgboost as xgb
import pickle
import csv

In [2]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parents[1]))
from Helper_Functions import common_utils
directory_to_extract_to = os.getcwd()

common_utils.load_data_from_one_drive(directory_to_extract_to, "recommendation_paths", "bankproduct_recommendation")

Extracting : 100%|██████████| 1/1 [00:01<00:00,  1.42s/it]


In [2]:
df= pd.read_csv('sliced.csv')

C:\Users\Fahad.A\AppData\Local\Temp\ipykernel_9048\3488184092.py:1: DtypeWarning: Columns (10,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('sliced.csv')


In [3]:
df.columns

Index(['Status_Dt', 'Cust_ID', 'Emp_Status', 'Cust_Ctry', 'Cust_Sex', 'Age',
       'Join_Dt', 'Is_New_Cust', 'Cust_Since', 'Relship_Status',
       'Lst_Dt_Primary_Cust', 'Cust_Type_Mth_Beg', 'Cust_Relship_Type_Mth_Beg',
       'Residence_Ctry', 'Is_Foreigner', 'Is_Spouse_Emp', 'Join_Channel',
       'Deceased_Flg', 'Address_Type', 'Cust_Province_Cd',
       'Cust_Province_Name', 'Cust_Active_Status', 'Gross_HHLD_Income',
       'Cust_Segment', 'Savings_Acct', 'Guarantees', 'Cur_Acct',
       'Derivative_Acct', 'Payroll_Acct', 'Junior_Acct', 'Mas_Particular_Acct',
       'Particular_Acct', 'Particular_Plus_Acct', 'Short_Term_Deposits',
       'Med_Term_Deposits', 'Long_Term_Deposits', 'e-Acct', 'Funds',
       'Mortgage', 'Pension1', 'Loans', 'Taxes', 'Credit_Card', 'Securities',
       'Home_Acct', 'Payroll', 'Pension2', 'Direct_Debit'],
      dtype='object')

In [4]:
df['Status_Dt'] = pd.to_datetime(df['Status_Dt'],format="%Y-%m-%d")

In [5]:
df['Year'] = df['Status_Dt'].dt.year
df['Monthly'] = df['Status_Dt'].dt.month
df['Day'] = df['Status_Dt'].dt.day

In [6]:
numeric_attribs = ['Age', 'Is_New_Cust', 'Cust_Since', 'Cust_Active_Status']
for attrib in numeric_attribs:
    #errors='coerce' - when value is invalid, set it to NAN
    df[attrib] = pd.to_numeric(df[attrib], errors='coerce')

In [7]:
df.drop('Is_Spouse_Emp', axis=1, inplace=True)
df.drop('Lst_Dt_Primary_Cust', axis=1, inplace=True)
df.drop('Address_Type', axis=1, inplace=True)
df.drop('Join_Dt', axis=1, inplace=True)
df.drop('Status_Dt', axis=1, inplace=True)
df.drop('Cust_ID', axis=1, inplace=True)
df.drop('Deceased_Flg', axis=1, inplace=True)
df.drop('Cust_Type_Mth_Beg', axis=1, inplace=True)
df.drop('Relship_Status', axis=1, inplace=True)
df.drop('Day', axis=1, inplace=True)
df.drop('Year', axis=1, inplace=True)
df.drop('Cust_Ctry', axis=1, inplace=True)
df.drop('Is_Foreigner', axis=1, inplace=True)

In [8]:
df.drop('Cust_Province_Cd', axis=1, inplace=True)

In [9]:
df.loc[df["Gross_HHLD_Income"].isnull(),"Gross_HHLD_Income"] = np.nanmedian(df.Gross_HHLD_Income)
df.loc[df["Cust_Segment"].isnull(),"Cust_Segment"] = '02 - PARTICULARES'
df.loc[df["Join_Channel"].isnull(),"Join_Channel"] = 'KHE'
df.loc[df["Cust_Relship_Type_Mth_Beg"].isnull(),"Cust_Relship_Type_Mth_Beg"] = 'I'
#df.loc[df["Cust_Province_Cd"].isnull(),"Cust_Province_Cd"] = 28.0
df.loc[df["Cust_Province_Name"].isnull(),"Cust_Province_Name"] = 'MADRID'
df.loc[df["Cust_Sex"].isnull(),"Cust_Sex"] = 'V'

In [10]:
numeric_column = ['Age', 'Cust_Active_Status', 'Gross_HHLD_Income', 'Cust_Since', 'Monthly']

In [11]:
df.columns

Index(['Emp_Status', 'Cust_Sex', 'Age', 'Is_New_Cust', 'Cust_Since',
       'Cust_Relship_Type_Mth_Beg', 'Residence_Ctry', 'Join_Channel',
       'Cust_Province_Name', 'Cust_Active_Status', 'Gross_HHLD_Income',
       'Cust_Segment', 'Savings_Acct', 'Guarantees', 'Cur_Acct',
       'Derivative_Acct', 'Payroll_Acct', 'Junior_Acct', 'Mas_Particular_Acct',
       'Particular_Acct', 'Particular_Plus_Acct', 'Short_Term_Deposits',
       'Med_Term_Deposits', 'Long_Term_Deposits', 'e-Acct', 'Funds',
       'Mortgage', 'Pension1', 'Loans', 'Taxes', 'Credit_Card', 'Securities',
       'Home_Acct', 'Payroll', 'Pension2', 'Direct_Debit', 'Monthly'],
      dtype='object')

In [12]:
c = df.columns
len(c)

37

In [13]:
c = df.columns
df[[c[12], c[36]]] = df[[c[36], c[12]]]
df.head()

,Emp_Status,Cust_Sex,Age,Is_New_Cust,Cust_Since,Cust_Relship_Type_Mth_Beg,Residence_Ctry,Join_Channel,Cust_Province_Name,Cust_Active_Status,...,Pension1,Loans,Taxes,Credit_Card,Securities,Home_Acct,Payroll,Pension2,Direct_Debit,Monthly
0,N,V,22,0.0,28,I,S,KHE,VALENCIA,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
1,N,H,27,0.0,28,A,S,KHE,MADRID,1.0,...,0,0,0,0,0,0,0.0,0.0,1,0
2,N,H,26,0.0,28,A,S,KFC,MADRID,1.0,...,0,0,0,0,0,0,1.0,1.0,0,0
3,N,H,26,0.0,28,I,S,KHE,MADRID,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
4,N,H,29,0.0,28,I,S,KHE,GRANADA,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0


In [14]:
col_list = list(df)
# use this handy way to swap the elements
col_list[12], col_list[36] = col_list[36], col_list[12]
# assign back, the order will now be swapped
df.columns = col_list
df.head()

,Emp_Status,Cust_Sex,Age,Is_New_Cust,Cust_Since,Cust_Relship_Type_Mth_Beg,Residence_Ctry,Join_Channel,Cust_Province_Name,Cust_Active_Status,...,Pension1,Loans,Taxes,Credit_Card,Securities,Home_Acct,Payroll,Pension2,Direct_Debit,Savings_Acct
0,N,V,22,0.0,28,I,S,KHE,VALENCIA,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
1,N,H,27,0.0,28,A,S,KHE,MADRID,1.0,...,0,0,0,0,0,0,0.0,0.0,1,0
2,N,H,26,0.0,28,A,S,KFC,MADRID,1.0,...,0,0,0,0,0,0,1.0,1.0,0,0
3,N,H,26,0.0,28,I,S,KHE,MADRID,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
4,N,H,29,0.0,28,I,S,KHE,GRANADA,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0


In [15]:
from sklearn import preprocessing

scale_list = []
normalise_list = []
data_skew = df.skew()

for i in numeric_column:
    if data_skew[i] >= -0.5 and data_skew[i] <= 0.5:
        scale_list.append(i)
    else:
        normalise_list.append(i)
print(scale_list)
print(normalise_list)

['Cust_Active_Status']
['Age', 'Gross_HHLD_Income', 'Cust_Since', 'Monthly']


C:\Users\Fahad.A\AppData\Local\Temp\ipykernel_9048\252316343.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data_skew = df.skew()


In [16]:
features = df[scale_list]
scaler = preprocessing.StandardScaler().fit(features.values)
features = scaler.transform(features.values)
df[scale_list] = features
df.head()

,Emp_Status,Cust_Sex,Age,Is_New_Cust,Cust_Since,Cust_Relship_Type_Mth_Beg,Residence_Ctry,Join_Channel,Cust_Province_Name,Cust_Active_Status,...,Pension1,Loans,Taxes,Credit_Card,Securities,Home_Acct,Payroll,Pension2,Direct_Debit,Savings_Acct
0,N,V,22,0.0,28,I,S,KHE,VALENCIA,-0.849968,...,0,0,0,0,0,0,0.0,0.0,0,0
1,N,H,27,0.0,28,A,S,KHE,MADRID,1.176515,...,0,0,0,0,0,0,0.0,0.0,1,0
2,N,H,26,0.0,28,A,S,KFC,MADRID,1.176515,...,0,0,0,0,0,0,1.0,1.0,0,0
3,N,H,26,0.0,28,I,S,KHE,MADRID,-0.849968,...,0,0,0,0,0,0,0.0,0.0,0,0
4,N,H,29,0.0,28,I,S,KHE,GRANADA,-0.849968,...,0,0,0,0,0,0,0.0,0.0,0,0


In [17]:
features = df[normalise_list]
scaler_1 = preprocessing.MinMaxScaler().fit(features.values)
features = scaler_1.transform(features.values)
df[normalise_list] = features
df.head()

,Emp_Status,Cust_Sex,Age,Is_New_Cust,Cust_Since,Cust_Relship_Type_Mth_Beg,Residence_Ctry,Join_Channel,Cust_Province_Name,Cust_Active_Status,...,Pension1,Loans,Taxes,Credit_Card,Securities,Home_Acct,Payroll,Pension2,Direct_Debit,Savings_Acct
0,N,V,0.123457,0.0,0.999772,I,S,KHE,VALENCIA,-0.849968,...,0,0,0,0,0,0,0.0,0.0,0,0
1,N,H,0.154321,0.0,0.999772,A,S,KHE,MADRID,1.176515,...,0,0,0,0,0,0,0.0,0.0,1,0
2,N,H,0.148148,0.0,0.999772,A,S,KFC,MADRID,1.176515,...,0,0,0,0,0,0,1.0,1.0,0,0
3,N,H,0.148148,0.0,0.999772,I,S,KHE,MADRID,-0.849968,...,0,0,0,0,0,0,0.0,0.0,0,0
4,N,H,0.166667,0.0,0.999772,I,S,KHE,GRANADA,-0.849968,...,0,0,0,0,0,0,0.0,0.0,0,0


In [18]:
df['Payroll'] = df['Payroll'].astype(int)
df['Pension2'] = df['Pension2'].astype(int)

In [19]:
encoder_list = ['Emp_Status', 'Cust_Sex', 'Cust_Relship_Type_Mth_Beg', 'Join_Channel', 'Cust_Segment', 'Residence_Ctry', 'Cust_Province_Name']

In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df.Emp_Status) 
df.Emp_Status = le.transform(df.Emp_Status)
le0 = LabelEncoder()
le0.fit(df.Cust_Sex) 
df.Cust_Sex = le0.transform(df.Cust_Sex)
le2 = LabelEncoder()
le2.fit(df.Cust_Relship_Type_Mth_Beg) 
df.Cust_Relship_Type_Mth_Beg = le2.transform(df.Cust_Relship_Type_Mth_Beg)
le3 = LabelEncoder()
le3.fit(df.Join_Channel) 
df.Join_Channel = le3.transform(df.Join_Channel)
le4 = LabelEncoder()
le4.fit(df.Cust_Segment) 
df.Cust_Segment = le4.transform(df.Cust_Segment)
le5 = LabelEncoder()
le5.fit(df.Residence_Ctry) 
df.Residence_Ctry = le5.transform(df.Residence_Ctry)
le7 = LabelEncoder()
le7.fit(df.Cust_Province_Name) 
df.Cust_Province_Name = le7.transform(df.Cust_Province_Name)

In [21]:
df.columns

Index(['Emp_Status', 'Cust_Sex', 'Age', 'Is_New_Cust', 'Cust_Since',
       'Cust_Relship_Type_Mth_Beg', 'Residence_Ctry', 'Join_Channel',
       'Cust_Province_Name', 'Cust_Active_Status', 'Gross_HHLD_Income',
       'Cust_Segment', 'Monthly', 'Guarantees', 'Cur_Acct', 'Derivative_Acct',
       'Payroll_Acct', 'Junior_Acct', 'Mas_Particular_Acct', 'Particular_Acct',
       'Particular_Plus_Acct', 'Short_Term_Deposits', 'Med_Term_Deposits',
       'Long_Term_Deposits', 'e-Acct', 'Funds', 'Mortgage', 'Pension1',
       'Loans', 'Taxes', 'Credit_Card', 'Securities', 'Home_Acct', 'Payroll',
       'Pension2', 'Direct_Debit', 'Savings_Acct'],
      dtype='object')

In [22]:
result = df.dtypes

print("Output:")
print(result)

Output:
Emp_Status                     int32
Cust_Sex                       int32
Age                          float64
Is_New_Cust                  float64
Cust_Since                   float64
Cust_Relship_Type_Mth_Beg      int32
Residence_Ctry                 int32
Join_Channel                   int32
Cust_Province_Name             int32
Cust_Active_Status           float64
Gross_HHLD_Income            float64
Cust_Segment                   int32
Monthly                      float64
Guarantees                     int64
Cur_Acct                       int64
Derivative_Acct                int64
Payroll_Acct                   int64
Junior_Acct                    int64
Mas_Particular_Acct            int64
Particular_Acct                int64
Particular_Plus_Acct           int64
Short_Term_Deposits            int64
Med_Term_Deposits              int64
Long_Term_Deposits             int64
e-Acct                         int64
Funds                          int64
Mortgage                      

In [23]:
target_dict = {'Savings_Acct':23, 'Guarantees':0, 'Cur_Acct':1,
       'Derivative_Acct':2, 'Payroll_Acct':3, 'Junior_Acct':4, 'Mas_Particular_Acct':5,
       'Particular_Acct':6, 'Particular_Plus_Acct':7, 'Short_Term_Deposits':8,
       'Med_Term_Deposits':9, 'Long_Term_Deposits':10, 'e-Acct':11, 'Funds':12,
       'Mortgage':13, 'Pension1':14, 'Loans':15, 'Taxes':16, 'Credit_Card':17, 'Securities':18,
       'Home_Acct':19, 'Payroll':20, 'Pension2':21, 'Direct_Debit':22}

In [24]:
df.shape

(1000000, 37)

In [25]:
target_cols = ['Guarantees', 'Cur_Acct',
       'Derivative_Acct', 'Payroll_Acct', 'Junior_Acct', 'Mas_Particular_Acct',
       'Particular_Acct', 'Particular_Plus_Acct', 'Short_Term_Deposits',
       'Med_Term_Deposits', 'Long_Term_Deposits', 'e-Acct', 'Funds',
       'Mortgage', 'Pension1', 'Loans', 'Taxes', 'Credit_Card', 'Securities',
       'Home_Acct', 'Payroll', 'Pension2', 'Direct_Debit', 'Savings_Acct']

In [26]:
arr = df.to_numpy()
arr.shape

(1000000, 37)

In [27]:
lst = arr.tolist()
#lst

In [28]:
len(lst)

1000000

In [29]:
x_1 = []
y_1 = []
for x  in lst:
    x_2 = []
    count = 37
    flag = 0
    for y in range(13,37):
        if x[y] == 1:
            val = y-13
            #print(val)
            y_1.append(val)
            x_1.append(x[0:13])
            #print(x[0:24])
            #x_1.append(x_2)
            flag = 1
            count -=1

    if flag == 0:
        x_1.append(x[0:13])
        #x_1.append(x_2)

    if count == 37:
        y_1.append(99)
    #print(x_2)

#print(x_1)



In [30]:
len(x_1)

1555283

In [31]:
train_X = np.array(x_1)
train_y = np.array(y_1)

In [32]:
train_X.shape

(1555283, 13)

In [33]:
train_y.shape

(1555283,)

In [34]:
print(np.bincount(train_y).argmax())

1


In [35]:
print(np.unique(train_y))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 99]


In [36]:
for i in range(1555283):
    if train_y[i] == 99:
        train_y[i] = 1

In [37]:
print(np.unique(train_y))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size = 0.25, random_state = 0)

In [39]:
y_train.shape

(1166462,)

In [40]:
X_train.shape

(1166462, 13)

In [41]:
def runXGB(train_X, train_y, seed_val=123):
	param = {}
	param['objective'] = 'multi:softprob'
	param['eta'] = 0.05
	param['max_depth'] = 6
	param['silent'] = 1
	param['num_class'] = 24
	param['eval_metric'] = "mlogloss"
	param['min_child_weight'] = 1
	param['subsample'] = 0.95
	param['colsample_bytree'] = 0.95
	param['seed'] = seed_val
	num_rounds = 100

	plst = list(param.items())
	xgtrain = xgb.DMatrix(train_X, label=train_y)
	model = xgb.train(plst, xgtrain, num_rounds)	
	return model

In [44]:
model = runXGB(X_train, y_train, seed_val=0)

[12:48:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [42]:
model = pickle.load(open('Model_XGB.pkl', 'rb'))

In [43]:
xgtest = xgb.DMatrix(X_test)

In [84]:
preds = model.predict(xgtest)
print(preds.shape)
preds = np.argsort(preds, axis=1)
print(preds.shape)
preds = np.fliplr(preds)[:,:3]
print(preds.shape)

(388821, 24)
(388821, 24)
(388821, 3)


In [85]:
target_cols = np.array(target_cols)

In [86]:
preds[90]

array([ 1, 22,  6], dtype=int64)

In [87]:
y_test.shape

(388821,)

In [88]:
hit = 0
miss = 0

In [89]:
res = []
for i in range(3):
    val = preds[91][i]
    res.append(target_cols[val])

res

['Cur_Acct', 'Particular_Acct', 'Taxes']

In [90]:
#pickle.dump(model, open('Model_XGB.pkl', 'wb'))

In [92]:
for i in range(388821):

    val = y_test[i]
    flag = 0
    for j in range(3):

        if val == preds[i][j]:
            hit += 1
            flag = 1
            break
    if flag == 0:
        miss += 1
print(hit)
print(miss)

293038
95784


In [52]:
del arr
del lst
del x_1
del x_2
del y_1
del train_X
del train_y

In [53]:
del X_test
del X_train
del y_train

In [54]:
del df

In [55]:
preds[388815:388820]

array([[ 1, 16,  3, 22, 11,  6,  7, 20],
       [ 1, 22,  6,  3, 11, 17, 16, 20],
       [ 1,  7, 16, 11,  6, 22,  3,  5],
       [ 1, 22, 11,  3, 21, 20, 10, 16],
       [ 1,  6, 22, 11,  3, 17, 21, 20]], dtype=int64)

In [64]:
res = []
for i in range(388811,388821):
    val = []
    for j in range(8):
        val1 = preds[i][j]
        val.append(target_cols[val1])
    
    res.append(val)

res

[['Cur_Acct',
  'Direct_Debit',
  'Payroll_Acct',
  'Mas_Particular_Acct',
  'Pension2',
  'Payroll',
  'e-Acct',
  'Taxes'],
 ['Cur_Acct',
  'e-Acct',
  'Taxes',
  'Payroll_Acct',
  'Direct_Debit',
  'Payroll',
  'Pension2',
  'Mas_Particular_Acct'],
 ['Cur_Acct',
  'e-Acct',
  'Direct_Debit',
  'Payroll_Acct',
  'Pension2',
  'Payroll',
  'Taxes',
  'Credit_Card'],
 ['Cur_Acct',
  'Taxes',
  'Payroll_Acct',
  'Direct_Debit',
  'e-Acct',
  'Payroll',
  'Pension2',
  'Mas_Particular_Acct'],
 ['Cur_Acct',
  'Taxes',
  'Payroll_Acct',
  'Direct_Debit',
  'e-Acct',
  'Particular_Acct',
  'Particular_Plus_Acct',
  'Payroll'],
 ['Cur_Acct',
  'Direct_Debit',
  'Particular_Acct',
  'Payroll_Acct',
  'e-Acct',
  'Credit_Card',
  'Taxes',
  'Payroll'],
 ['Cur_Acct',
  'Particular_Plus_Acct',
  'Taxes',
  'e-Acct',
  'Particular_Acct',
  'Direct_Debit',
  'Payroll_Acct',
  'Mas_Particular_Acct'],
 ['Cur_Acct',
  'Direct_Debit',
  'e-Acct',
  'Payroll_Acct',
  'Pension2',
  'Payroll',
  'Long_Te

In [65]:
res[4]

['Cur_Acct',
 'Taxes',
 'Payroll_Acct',
 'Direct_Debit',
 'e-Acct',
 'Particular_Acct',
 'Particular_Plus_Acct',
 'Payroll']

In [66]:
res[3]

['Cur_Acct',
 'Taxes',
 'Payroll_Acct',
 'Direct_Debit',
 'e-Acct',
 'Payroll',
 'Pension2',
 'Mas_Particular_Acct']

In [67]:
res[2]

['Cur_Acct',
 'e-Acct',
 'Direct_Debit',
 'Payroll_Acct',
 'Pension2',
 'Payroll',
 'Taxes',
 'Credit_Card']

In [68]:
res[1]

['Cur_Acct',
 'e-Acct',
 'Taxes',
 'Payroll_Acct',
 'Direct_Debit',
 'Payroll',
 'Pension2',
 'Mas_Particular_Acct']

In [69]:
res[5]

['Cur_Acct',
 'Direct_Debit',
 'Particular_Acct',
 'Payroll_Acct',
 'e-Acct',
 'Credit_Card',
 'Taxes',
 'Payroll']

In [74]:
res[9]

['Cur_Acct',
 'Payroll_Acct',
 'Taxes',
 'Direct_Debit',
 'e-Acct',
 'Mas_Particular_Acct',
 'Pension2',
 'Payroll']

Junior_Acct
Cur_Acct
Particular_Acct
Pension1
Savings_Acct
Guarantees
Home_Acct
Payroll
Med_Term_Deposits
Mortgage
Long_Term_Deposits
Particular_Plus_Acct
Short_Term_Deposits
Pension2
Loans
Credit_Card
